In [1]:
import pandas as pd
import pickle
import numpy as np

In [2]:
def pickle_reader(path):
    emb_input = pd.read_pickle(path)
    return emb_input
    
def dict_to_matrix(input_dict, names):
    emb_matrix = np.array([input_dict[i] for i in names])
    return emb_matrix

#def merge_emb_counts(emb_i)

In [3]:
path = '/Users/leandro/Desktop/ai_data/data/embeddings_v2.pickle'
emb_input = pickle_reader(path)
emb_input

{'NeSL-1DF0001444': array([ 0.03649043, -0.05133745, -0.02149544, ...,  0.05933682,
         0.02960958, -0.00366774], dtype=float32),
 'L1-13_DRDF0002999': array([-0.00565536,  0.04626307,  0.06430495, ...,  0.05257737,
         0.10679311,  0.05897566], dtype=float32),
 'SurSur-1_49DF0280846': array([ 0.01012634,  0.05808156,  0.03795375, ...,  0.05786201,
        -0.02371022, -0.00337778], dtype=float32),
 'Tx1-10_DRDF0003770': array([-0.00124524, -0.04418022, -0.02952355, ...,  0.03803641,
         0.00997921, -0.01272242], dtype=float32),
 'L1_Mur1_orf2DF0001882': array([-0.00047515,  0.05268906,  0.02921422, ...,  0.05128015,
        -0.02365779, -0.00949796], dtype=float32),
 'Tx1-1_DRDF0003950': array([-0.00318524, -0.04321994, -0.03308535, ...,  0.04633515,
         0.01255036, -0.01444965], dtype=float32),
 'ParHer-6_210DF0279964': array([ 0.01164113,  0.06202403,  0.03903304, ...,  0.05591419,
        -0.02466295, -0.00379632], dtype=float32),
 'SurSur-4_851DF0280848': array

In [5]:
names = emb_input.keys()
emb_matrix = dict_to_matrix(emb_input, names)
emb_matrix

array([[ 0.03649043, -0.05133745, -0.02149544, ...,  0.05933682,
         0.02960958, -0.00366774],
       [-0.00565536,  0.04626307,  0.06430495, ...,  0.05257737,
         0.10679311,  0.05897566],
       [ 0.01012634,  0.05808156,  0.03795375, ...,  0.05786201,
        -0.02371022, -0.00337778],
       ...,
       [ 0.00024575, -0.01960255, -0.02111155, ...,  0.05700654,
         0.01444513,  0.00038578],
       [ 0.01972299,  0.01057831, -0.01364452, ...,  0.02833491,
        -0.00725596,  0.00280471],
       [ 0.01875487,  0.07443045,  0.043456  , ...,  0.05848584,
        -0.02183677, -0.01166718]], dtype=float32)

In [11]:
emb_matrix.dtype.names

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import umap
import pickle
import scipy.sparse as sparse
from scipy.io import mmwrite
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import scanpy as sc
import anndata


# Create matrix of embeddings
matrix = pd.DataFrame(pickle_data)
matrix.index = ["dimension" + str(i) for i in range(1, matrix.shape[0]+1)]

# Visualize the IDs
print(matrix.columns.values[:6])
print(ORF_counts["ID"][:6])

# Match IDs from the embeddings data and the ORF counts data
subset_matrix = matrix.loc[:, matrix.columns.isin(ORF_counts["ID"].values)]

# Create AnnData object
adata = anndata.AnnData(X=subset_matrix.values, obs=pd.DataFrame(index=subset_matrix.columns))

# Add counts
ID_embeddings = pd.DataFrame(index=adata.obs.index)
ID_embeddings_with_counts = pd.merge(ID_embeddings, ORF_counts, left_index=True, right_on="ID", how="left").sort_values("ID")
assert all(ID_embeddings.index == ID_embeddings_with_counts["ID"])
adata.obs["ORF_all_counts"] = ID_embeddings_with_counts["All"].values
adata.obs["ORF_nonredundant_counts"] = ID_embeddings_with_counts["Non_redundant"].values

# Scale data to use for PCA
sc.pp.scale(adata)

# Run PCA and visualize data
pca = PCA(n_components=subset_matrix.shape[1])
adata.obsm["X_pca"] = pca.fit_transform(adata.X)
sc.pp.neighbors(adata, n_neighbors=10, use_rep="X_pca")
sc.tl.umap(adata, n_components=2, metric="cosine")

# Import Seurat library
import scanpy as sc

# Load data
adata = sc.read_h5ad('your_file.h5ad')

# Run PCA
sc.pp.pca(adata, use_highly_variable=True)

# Visualize PCA results
sc.pl.pca(adata, color='ORF_nonredundant_counts')

# Select PCs for UMAP
sc.pl.pca_variance_ratio(adata)
sc.tl.pca(adata, n_comps=21)
sc.pl.pca_variance_ratio(adata, log=True)

# Run UMAP
sc.pp.neighbors(adata, n_neighbors=30, n_pcs=21)
sc.tl.umap(adata)

# Visualize UMAP results
sc.pl.umap(adata, color='ORF_nonredundant_counts')


Note that some of the code in the original R script, such as saving plots to a PDF file, will need to be modified or removed for use in Python.

In [ ]:
# Import Seurat library
import scanpy as sc
import matplotlib.pyplot as plt

# Define a function to load data
def load_data(filename):
    return sc.read_h5ad(filename)

# Define a function to run PCA
def run_pca(data):
    sc.pp.pca(data, use_highly_variable=True)
    return data

# Define a function to visualize PCA results
def plot_pca(data):
    sc.pl.pca(data, color='ORF_nonredundant_counts')

# Define a function to select PCs for UMAP
def select_pcs(data):
    sc.tl.pca(data, n_comps=21)
    sc.pl.pca_variance_ratio(data, log=True)
    return data

# Define a function to run UMAP
def run_umap(data):
    sc.pp.neighbors(data, n_neighbors=30, n_pcs=21)
    sc.tl.umap(data)
    return data

# Define a function to visualize UMAP results
def plot_umap(data):
    sc.pl.umap(data, color='ORF_nonredundant_counts')

# Load data
adata = load_data('your_file.h5ad')

# Run PCA
adata = run_pca(adata)

# Visualize PCA results
plot_pca(adata)

# Select PCs for UMAP
adata = select_pcs(adata)

# Run UMAP
adata = run_umap(adata)

# Visualize UMAP results
plot_umap(adata)

# Save UMAP plot as a PDF file
plt.savefig('UMAP_1_2_plot.pdf', format='pdf', dpi=300)

In [ ]:
This code defines functions for loading data, running PCA, visualizing PCA results, selecting PCs for UMAP, running UMAP, visualizing UMAP results, and saving a UMAP plot as a PDF file.
The main code loads the data, runs PCA, visualizes PCA results, selects PCs for UMAP, runs UMAP, visualizes UMAP results, and saves a U